## CIFAR10实战

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)
    
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.1.0
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
def preprocess(x, y):
    """x is a simple image, not a batch"""
    # [0 -- 255] => [-1 -- 1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

batchsz = 128
# [50k, 32, 32, 3]  [10k, 1]
(x, y), (x_val, y_val) = datasets.cifar10.load_data()
# 去除维度数为 1 的维，进行压缩
y = tf.squeeze(y)    
y_val = tf.squeeze(y_val)
y = tf.one_hot(y, depth=10)    # [50k, 10]
y_val = tf.one_hot(y_val, depth=10)    # [10k, 10]
print('datasets:', x.shape,x_val.shape, y.shape, y_val.shape, x.min(), x.max())


datasets: (50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10) 0 255


In [3]:
# 数据预处理
train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batchsz)

sample = next(iter(train_db))
print('batch: ', sample[0].shape, sample[1].shape)
sample_test = next(iter(test_db))
print('batch: ', sample_test[0].shape, sample_test[1].shape)

batch:  (128, 32, 32, 3) (128, 10)
batch:  (128, 32, 32, 3) (128, 10)


In [4]:
# # 定义监听器
# current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
# log_dir = '/home/kukafee/workspace/TensorBoard/' + current_time
# summary_writer = tf.summary.create_file_writer(log_dir)

In [5]:
# 自定义层
class MyDense(layers.Layer):
    # to replace standard layeras.Dense()
    def __init__(self, inp_dim, outp_dim):
        # 调用父类的初始化方法
        super(MyDense, self).__init__()
        self.kernel = self.add_variable('w', [inp_dim, outp_dim])
#         self.bias = self.add_variable('b', [outp_dim])
        
    def call(self, inputs, training=None):
        x = inputs @ self.kernel
        return x
    
# 自定义网络
class MyNetwork(keras.Model):
    def __init__(self):
        super(MyNetwork, self).__init__()
        
        self.fc1 = MyDense(32*32*3, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
        
    def call(self, inputs, training=None):
        # inputs [b, 32, 32, 3]
        x = tf.reshape(inputs, [-1, 32*32*3])    # [b, 32*32*3]
        
        x = self.fc1(x)    # [b, 32*32*3] => [b, 256]
        x = tf.nn.relu(x)
        x = self.fc2(x)    # [b, 256] => [b, 128]
        x = tf.nn.relu(x)
        x = self.fc3(x)    # [b, 128] => [b, 64]
        x = tf.nn.relu(x)
        x = self.fc4(x)    # [b, 64] => [b, 32]
        x = tf.nn.relu(x)
        x = self.fc5(x)    # [b, 32] => [b, 10]
        
        return x

# 实例化神经网络
network = MyNetwork()
# 装配神经网络实例: optimizer 指定优化器；loss 指定损失函数； metrics 指定在训练的同时计算精确度
network.compile(optimizer=optimizers.Adam(lr=1e-3),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
# epochs 指定训练时代数；validation_data指定在训练时进行测试所使用的测试集
# validation_freq 指定每进行多少个 epochs 测试一次
network.fit(train_db, epochs=20, validation_data=test_db, validation_freq=1)
# 训练完成后测试
network.evaluate(test_db)
# save network， 其中保存的文件名自定义，文件名的后缀名也是自定义的，不会影响格式
network.save_weights('/home/kukafee/workspace/save_model/weights6174.ckpy')
del network
print('saved to "/home/kukafee/workspace/save_model/weights6174.ckpy"')

network = MyNetwork()
network.compile(optimizer=optimizers.Adam(lr=1e-3),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
network.load_weights('/home/kukafee/workspace/save_model/weights6174.ckpy')
print('Loaded weights from file.')
network.evaluate(test_db)           

Epoch 1/20
391/391 [==============================] - 3s 7ms/step - loss: 1.7201 - accuracy: 0.3478 - val_loss: 1.5662 - val_accuracy: 0.4497
Epoch 2/20
391/391 [==============================] - 2s 6ms/step - loss: 1.5086 - accuracy: 0.4577 - val_loss: 1.4974 - val_accuracy: 0.4770
Epoch 3/20
391/391 [==============================] - 2s 6ms/step - loss: 1.4001 - accuracy: 0.4981 - val_loss: 1.4592 - val_accuracy: 0.4899
Epoch 4/20
391/391 [==============================] - 2s 6ms/step - loss: 1.3103 - accuracy: 0.5308 - val_loss: 1.4362 - val_accuracy: 0.5027
Epoch 5/20
391/391 [==============================] - 2s 6ms/step - loss: 1.2333 - accuracy: 0.5582 - val_loss: 1.4471 - val_accuracy: 0.5013
Epoch 6/20
391/391 [==============================] - 2s 6ms/step - loss: 1.1648 - accuracy: 0.5858 - val_loss: 1.4852 - val_accuracy: 0.4973
Epoch 7/20
391/391 [==============================] - 2s 6ms/step - loss: 1.0972 - accuracy: 0.6083 - val_loss: 1.5080 - val_accuracy: 0.4993
Epoch 

[2.29873543449595, 0.4811]